## 1. Import raw dataset

In [1]:
import pandas as pd
import numpy as np
import json

# import ast
# from pandas.io.json import json_normalize

/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


In [2]:
# run 'json_to_csv.py' script to convert all json files into csv files of the same name
# nested dictionaries is flatterned during the conversion and both parent and nested key, value pairs are extracted

%run -i 'json_to_csv.py' 'dataset/business.json'
%run -i 'json_to_csv.py' 'dataset/user.json'
%run -i 'json_to_csv.py' 'dataset/review.json'
%run -i 'json_to_csv.py' 'dataset/tip.json'
%run -i 'json_to_csv.py' 'dataset/checkin.json'

In [3]:
# import all five csv files as Pandas dataframe

business = pd.read_csv('dataset/business.csv')  # contains business data including location data, attributes and categories
user = pd.read_csv('dataset/user.csv') # contains users data including the user's friend mapping and all the metadata associated with the user
review = pd.read_csv('dataset/review.csv') # contains full review text data including the user_id that wrote the review and the business_id the review is written for
tip = pd.read_csv('dataset/tip.csv') # tips written by a user on a business, tips are shorter than reviews and tend to convey quick suggestions
checkin = pd.read_csv('dataset/checkin.csv') # checkins on a business

## 2. Data cleanup

### 2.1 dataframe 'business'

In [4]:
# filter to US business only by:
# 1. filter by postal_code following the American 5-digit zip code format
# 2. filter by state abbreviation

ziplen = business.postal_code.astype(str).apply(len) # drop entries where postal_code is not 5-digit from business
business = business[ziplen == 5]

list_of_states = ['AL','AK','AR','AS','AZ','CA','CO','CT','DC','DE','FL','FM','GA','GU','HI','IA','ID','IL','IN','KS','KY','LA','MA','MD','ME','MH','MI','MN','MO','MS','MT','NC','ND','NE','NH','NJ','NM','NV','NY','MP','OH','OK','OR','PA','PR','PW','RI','SC','SD','TN','TX','UT','VA','VI','VT','WA','WI','WV','WY']
business = business[business.state.isin(list_of_states)]

In [5]:
# deal with missing city information
business.loc[58688,'city'] = 'Yuma' # add missing city info
business.loc[111036,'city'] = 'Raleigh' # add missing city info

In [6]:
# business category

from collections import defaultdict

category = defaultdict(int)

for idx, row in business.iterrows():
    if row['categories'] is not np.nan: 
        entries = row['categories'].split(',')
        for entry in entries: 
            entry = entry.strip().lower()
            category[entry] += 1

print(len(category))
sorted_cate = sorted(category.items(), key=lambda x: x[1], reverse=True)
print(sorted_cate[:10])

1264
[('restaurants', 34136), ('shopping', 23148), ('food', 17178), ('home services', 17169), ('beauty & spas', 14604), ('health & medical', 14050), ('local services', 11031), ('automotive', 10843), ('nightlife', 8441), ('event planning & services', 7470)]


As shown, the 'business' dataset contains many businesses categories (a total of 1263), many of which are not restaurants. For instances, there are 'shopping', 'health & medical', 'automotive', etc. For this project, the focus is on restaurants, businesses are filtered by their 'categories' and non-restaurant related businesses are removed.

In [7]:
# remove business types other than restaurants

def rmv_non_restaurant_busi(df, yes=[], no=[]): 
    
    """
    remove businesses other than restaurants based on the information provided under business 'categories'
    ---
    input:
    1. df: dataframe to be cleaned up
    2. yes: additional list of strings for identifying restaurants
    3. no: additional list of strings for identifying non-restaurant busineses
    note: 'yes' and 'no' are optional, if provided, an union of the default set and the provided list will be computed and used. 
    ---
    output:
    cleaned up dataframe containing only restaurant-based businesses
    """

    yes = set(['fast food','sandwiches','caterers','deserts','burgers']).union(set(yes))
    no = set(['shopping','home services','beauty & spas','hair salons','health & medical', 'local services','automotive',
              'fitness & instruction','transportation','airlines','pets','active life','professional services','banks & credit unions'
             'insurance','financial services','education','public services & government','session photography','car rental',
             'photographers','churches','religious organizations','tours','doctors','massage','pharmacy','drugstores']).union(set(no))
    
    idx_to_drop = []
    for idx, row in df.iterrows():
        if row['categories'] is not np.nan: 
            entries = row['categories'].split(',')
            for entry in entries:
                entry = entry.strip().lower()
                if entry in yes: 
                    break
                if entry in no:
                    idx_to_drop.append(idx)
                    break
        elif row['attributes.HairSpecializesIn'] is not np.nan:  # if 'categories' is not available, use 'attributes' to filter
            idx_to_drop.append(idx) 
                    
    return df.drop(index=idx_to_drop)

print("# of businesses of all categories:", len(business))
business = rmv_non_restaurant_busi(business)
print("# of businesses related to restaurants:", len(business))

# of businesses of all categories: 138757
# of businesses related to restaurants: 47554


In [8]:
# reset index
business.reset_index(drop=True, inplace=True)
print(business.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 47554 entries, 0 to 47553
Data columns (total 61 columns):
longitude                                47554 non-null float64
attributes.BikeParking                   30187 non-null object
attributes.AgesAllowed                   251 non-null object
attributes.BusinessParking               37191 non-null object
hours.Wednesday                          34191 non-null object
latitude                                 47553 non-null float64
categories                               47138 non-null object
hours.Sunday                             29570 non-null object
attributes.OutdoorSeating                33107 non-null object
attributes.Open24Hours                   288 non-null object
attributes.DietaryRestrictions           92 non-null object
postal_code                              47554 non-null object
attributes.RestaurantsTableService       27011 non-null object
hours.Tuesday                            33718 non-null object
attributes.Whe

### Summary of dataframe 'business' after cleaning:
note: dataframe 'business' also contains columns resulting from unpacking nested dictionaries under the 'attributes' and 'hours' columns, those 'child' columns feature column names starting with either 'attributes.' or 'hours.'. Below is a quick summary of only the 'parent' columns: <br>

1) business_id: no NaN, all business_ids are of the same length of 22 characters, and are case-sensitive. No duplication is found <br>
2) name: no NaN<br>

(location-related info from 3) to 8)) <br>
    3) address: there are 1.9% NaNs, but it's ok since postal_code and coordinates are used mostly instead of address<br>
    4) city: no NaN<br>
    5) postal_code: no NaN. Postal_codes do not match the American 5-digit format has been removed<br>
    6) state: no NaN<br>
    7) lattitude and 8) longitude: only 1 entry with missing lattitude. For this entry, postal_code info is available;<br>

9) stars: no NaN, all star ratings take values from 1.0, 1.5, 2.0, 2.5, 3.0, 3.5, 4.0, 4.5, 5.0<br>
10) review_count: no NaN, review counts range between 3 and 7968<br>
11) is_open: no NaN, integer taking values of 0 or 1 for closed (27.3%) or open (72.7%), respectively<br>
12) neighborhood: significant NaNs (66.7%), the top 5 neighborhoods are 'Westside','Southeast','Spring Valley','The Strip','downtown'. The neighborhood information will not be used as location information. Instead, it will be treated as one of the business features for NLP analysis<br>
13) attributes: contains some NaN (3.9%), all with subfeatures shown under column names featuring 'attributes.'<br>
    -most subfeatures are categorical with either True or False binary entries or a few categorical values;<br>
    -six subfeatures still contain nested dictionaries, these subfeatures are 'attributes.businessParking','attributes.Ambience','attributes.BestNights','attributes.GoodforMeal','attributes.DietaryRestrictions' and 'attributes.Music';<br>
14) categories: has a few NaNs (0.9%), values are strings contains phrases (comma separated) describing categories, such as 'Burgers','Mexican','Bar', etc. can be extracted as restaurant cuisines or other features<br>
15) hours: contains NaNs (26.8%), all with subfeatures shown under columns names featuring 'hours.' <br>
    -all subfeatures are day of the week from 'Monday' to 'Sunday', with string type values indicating the operating hours<br>

### 2.2 dataframe 'user'

In [9]:
# remove the one entry having the average_stars of 0.0, this user only has this one entry and the corresponding review is invalid ('nan')
user = user[user.average_stars != 0]
user.reset_index(inplace=True, drop=True)

print(user.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1518168 entries, 0 to 1518167
Data columns (total 22 columns):
compliment_funny      1518168 non-null int64
user_id               1518168 non-null object
compliment_cool       1518168 non-null int64
elite                 1518168 non-null object
useful                1518168 non-null int64
compliment_hot        1518168 non-null int64
compliment_more       1518168 non-null int64
compliment_profile    1518168 non-null int64
name                  1517693 non-null object
funny                 1518168 non-null int64
cool                  1518168 non-null int64
compliment_list       1518168 non-null int64
friends               1518168 non-null object
yelping_since         1518168 non-null object
review_count          1518168 non-null int64
compliment_note       1518168 non-null int64
fans                  1518168 non-null int64
compliment_plain      1518168 non-null int64
compliment_writer     1518168 non-null int64
compliment_photos     15181

### Summary of dataframe 'user' after cleaning:
1) user_id: no NaNs, similar to business_id, all user_ids are of the same length of 22 characters, and are case-sensitive. No duplication is found.<br>
2) name: a few NaNs (0.03%). For all those, user_ids are available and user_id will be used in most cases.<br>
3) elite: no NaNs, a list of the years the user was an elite member. Most users (95.6%) has 'None' as the value under column 'elite', indicating that they have never been an elite member. In general, elite members are very active Yelp users will frequent activities and many insightful reviews/tips.<br>
4) yelping_since: no NaNs, string formatted as YYYY-MM-DD indicating the date the user joined Yelp. The dates range between 2004-10-12 and 2018-07-02.<br>
5) review_count: no NaNs,integer indicating the number of reviews the user has written, value ranges between 0 and 12723.<br>
6) average_stars: no NaNs, takes any float number between 1.00 and 5.00.<br>
7) useful: no NaNs, integer indicates the number of useful votes sent by the user. Value ranges between 0 and 258479,with 0 being the most common value.<br>
8) funny:  no NaNs, integer indicates the number of useful votes sent by the user. Value ranges between 0 and 242120,with 0 being the most common value.<br>
9) cool: no NaNs, integer indicates the number of useful votes sent by the user. Value ranges between 0 and 255909,with 0 being the most common value.<br>
10) fans: no NaNs, integer indiates the number of fans the user has. Value ranges between 0 and 8665, with 0 being the most common value. <br>
11) compliment_ : all have no NaNs, all integers indicating the number of various types of compliments received by the users.<br>

### 2.3 dataframe 'review'

In [10]:
# convert all values in the 'text' column to string type
review['text'] = review.text.astype(str)

# remove the one entry with star rating of 0 under 'star' column and no actual review ('nan' under 'text' column)
review = review[review.stars != 0]
review.reset_index(inplace=True, drop=True)

# correct the values under columns 'useful' and 'cool' for review_id 3445401 from -1 to 0, since values should be non-negative
review.loc[3445401, ['cool','useful']] = 0

# replace the '\r' in a few review text with '\n\n' because having only '\r' causes problem when writing to and importing from csv files
review.loc[3009218,'text']=repr(review.loc[3009218,'text']).replace('\r','\n\n')
review.loc[5507254,'text']=repr(review.loc[5507254,'text']).replace('\r','\n\n')

print(review.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5996995 entries, 0 to 5996994
Data columns (total 9 columns):
text           object
funny          int64
business_id    object
stars          int64
cool           int64
user_id        object
date           object
useful         int64
review_id      object
dtypes: int64(4), object(5)
memory usage: 411.8+ MB
None


### Summary of dataframe 'review' after cleaning:
1) review_id: no NaNs, similar to user_id and business_id, all review_ids are of the same length of 22 characters, and are case-sensitive. No duplication is found. <br>
2) user_id: no NaNs, all of the same length of 22 characters, case-sensitive. A total of 1518169 unique user_ids are included in the dataframe 'review', consistent with the total number of user_ids in the 'user' dataframe.<br>
3) business_id: no NaNs, all of the same length of 22 characters, case-sensitive. A total of 188593 unique business_ids are included in the dataframe 'review', more than the total number of business_ids in the 'business' dataframe.<br>
4) stars: no NaNs, integer indicating the star rating, takes discrete values of 1, 2, 3, 4 and 5<br>
5) text: no NaNs and no empty entries, strings of the actual reviews, with length ranging from 1 to 5000.<br>
5) date: no NaNs, string of length 10 formatted as YYYY-MM-DD, dates ranges from 2004-10-12 to 2018-07-02.<br>
6) useful: no NaNs, integer, the number of useful votes the review received, values range from 0 to 1234.<br>
7) funny: no NaNs, integer, the number of useful votes the review received, values range from 0 to 505.<br>
8) cool: no NaNs, integer, the number of useful votes the review received, values range from 0 to 991.<br>

### 2.4 dataframe 'tip'

In [11]:
# drop entries with 'NaN' under 'text' column
tip = tip[tip.text.notnull()]
tip.reset_index(inplace=True, drop=True)

print(tip.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1185344 entries, 0 to 1185343
Data columns (total 5 columns):
likes          1185344 non-null int64
text           1185344 non-null object
business_id    1185344 non-null object
date           1185344 non-null object
user_id        1185344 non-null object
dtypes: int64(1), object(4)
memory usage: 45.2+ MB
None


### Summary of dataframe 'tip' after cleaning:
1) user_id: no NaNs, all of the same length of 22 characters, case-sensitive. A total of 304869 unique user_ids are included in the dataframe 'tip', less than the total number of user_ids in the 'user' dataframe.<br>
2) business_id: no NaNs, all of the same length of 22 characters, case-sensitive. A total of 121526 unique business_ids are included in the dataframe 'tip', less than the total number of business_ids in the 'business' dataframe.<br>
3) text: no NaNs and no empty entries, strings of the actual tips, with length ranging from 1 to 500.<br>
4) date: no NaNs, string of length 10 formatted as YYYY-MM-DD, dates ranges from 2009-04-15 to 2018-07-02.<br>
5) likes: no NaNs, integer indicating the number of likes the tip received, value ranges from 0 to 15.<br>

### 2.5 dataframe 'checkin'

In [12]:
# sort the columns by column names
ordered_names = sorted(checkin.columns)
checkin = checkin[ordered_names]
print(checkin.columns)

# add a new column named 'total_count' containing the sum of all checkins at all times (sum of all values under the nested columns with names starting with 'time.')
checkin['total_count'] = checkin.loc[:,'time.Fri-0':'time.Wed-9'].sum(axis=1).astype(int)

print(checkin.info())

Index(['business_id', 'time', 'time.Fri-0', 'time.Fri-1', 'time.Fri-10',
       'time.Fri-11', 'time.Fri-12', 'time.Fri-13', 'time.Fri-14',
       'time.Fri-15',
       ...
       'time.Wed-21', 'time.Wed-22', 'time.Wed-23', 'time.Wed-3', 'time.Wed-4',
       'time.Wed-5', 'time.Wed-6', 'time.Wed-7', 'time.Wed-8', 'time.Wed-9'],
      dtype='object', length=170)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 157075 entries, 0 to 157074
Columns: 171 entries, business_id to total_count
dtypes: float64(168), int64(1), object(2)
memory usage: 204.9+ MB
None


### Summary of dataframe 'checkin' after cleaning:
note: dataframe 'checkin' also contains columns resulting from unpacking nested dictionaries under column 'time', those nested columns feature column names starting with 'time.'. Below is a quick summary of only the parent columns, 'business_id' and 'time'. In addition, a column 'total_count' containing the sum of all checkins at all times is computed and added.

1) business_id: no NaNs, all business_ids are of the same length of 22 characters, and are case-sensitive. No duplication is found.<br>
2) time: no NaNs, parent columns with nested dictionaries containing checkin counts (value) under all times (key).<br>
3) total_count: no NaNs, integer indicating the sum of all checkins at all times for the business_id, values range from 1 to 138477.<br> 

## 3. Output cleaned dataframes to csv files

In [13]:
business.to_csv(path_or_buf='business_clean.csv',index=False)
user.to_csv(path_or_buf='user_clean.csv',index=False)
review.to_csv(path_or_buf='review_clean.csv',index=False)
tip.to_csv(path_or_buf='tip_clean.csv',index=False)
checkin.to_csv(path_or_buf='checkin_clean.csv',index=False)